<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/LTIMindTree2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Given a dataset of monthly sales records with salespeople names and their regions, calculate the month with the highest sales for
each region using window functions and the max() function. Ensure that the result includes the region name, month, and sales value.
Consider sales fluctuations, and the dataset should contain multiple records for each region to test windowing correctly.


In [10]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.window import Window

spark=SparkSession.builder.appName("sales").getOrCreate()

data = [("Amit", "North", "Jan", 12000), ("Rajesh", "North", "Feb", 15000),
        ("Sunita", "North", "Mar", 11000), ("Meena", "South", "Jan", 17000),
        ("Ravi", "South", "Feb", 20000), ("Priya", "South", "Mar", 18000),
        ("Suresh", "East", "Jan", 10000), ("Vishal", "East", "Feb", 22000),
        ("Akash", "East", "Mar", 21000), ("Anjali", "West", "Jan", 15000),
        ("Deepak", "West", "Feb", 13000), ("Nidhi", "West", "Mar", 17000), ]

columns = ["Salesperson", "Region", "Month", "Sales"]

df=spark.createDataFrame(data,columns)

df.show()

df1=df.groupBy('Region','Month').agg(sum('Sales').alias('sales_value'))

df1.show()

window_spec=Window.partitionBy('Region').orderBy(col('sales_value').desc())

df2=df1.withColumn('row_num',dense_rank().over(window_spec)).filter(col('row_num')==1)\
      .select("Region",'Month',"sales_value")

df2.show()

+-----------+------+-----+-----+
|Salesperson|Region|Month|Sales|
+-----------+------+-----+-----+
|       Amit| North|  Jan|12000|
|     Rajesh| North|  Feb|15000|
|     Sunita| North|  Mar|11000|
|      Meena| South|  Jan|17000|
|       Ravi| South|  Feb|20000|
|      Priya| South|  Mar|18000|
|     Suresh|  East|  Jan|10000|
|     Vishal|  East|  Feb|22000|
|      Akash|  East|  Mar|21000|
|     Anjali|  West|  Jan|15000|
|     Deepak|  West|  Feb|13000|
|      Nidhi|  West|  Mar|17000|
+-----------+------+-----+-----+

+------+-----+-----------+
|Region|Month|sales_value|
+------+-----+-----------+
| North|  Feb|      15000|
| South|  Jan|      17000|
| South|  Mar|      18000|
| North|  Jan|      12000|
| South|  Feb|      20000|
| North|  Mar|      11000|
|  West|  Feb|      13000|
|  East|  Jan|      10000|
|  East|  Mar|      21000|
|  West|  Mar|      17000|
|  West|  Jan|      15000|
|  East|  Feb|      22000|
+------+-----+-----------+

+------+-----+-----------+
|Region|Mon